# 2_data_preparation_suto

**Purpose:** State the purpose of the notebook.

**Methodology:** Quickly describe assumptions and processing steps.

**Results:** Describe and comment the most important results.

---

**Suggested next steps**

- [ ] State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [99]:
%matplotlib inline

import os
import pickle
from typing import List

# Data manipulation
from feature_engine.encoding import OneHotEncoder
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
import pandas as pd
import numpy as np

# Visualizations
import matplotlib as plt
from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline
from pypmml import Model

os.chdir('../')
from src.utils.data_describe import breve_descricao, serie_nulos, cardinalidade
os.chdir('./notebooks/')

# Options for pandas
# pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.options.display.max_rows = 120

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Custom functions

In [3]:
def create_time_dif_columns(df: pd.DataFrame, lst: List) -> pd.DataFrame:
    df_temp = df.copy()
    lst_temp = []

    for attribute in lst:
        column = attribute + '_YrSold'
        df_temp[column] = df_temp['YrSold'] - df_temp[attribute]
        df_temp.drop(columns=attribute, inplace=True)
        df_temp[[column]] = df_temp[[column]].fillna(999)
        df_temp[[column]] = df_temp[[column]].astype(float)
        
    return df_temp


def prepare_baseline(df: pd.DataFrame) -> pd.DataFrame:
    lst_area = [
        'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
        '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea'
    ]
    
    lst_columns_null = ['PoolQC', 'MiscFeature', 'Alley', 'Fence']
    
    lst_time = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
    
    lst_final = [
        'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
        '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF',
        'OpenPorchSF', 'PoolArea', 'YrSold', 'YearBuilt_YrSold', 'YearRemodAdd_YrSold',
        'GarageYrBlt_YrSold'
    ]
    
    df_output = df.copy()
    df_output.drop(columns=lst_columns_null, inplace=True)

    lst_final_columns = lst_area.copy()
    lst_final_columns.extend(lst_time)
    lst_final_columns.append('YrSold')
    
    if 'SalePrice' in df.columns:
        lst_final_columns.append('SalePrice')
        lst_final.append('SalePrice')

    df_output = df_output[lst_final_columns]
    df_output = create_time_dif_columns(df=df_output, lst=lst_time)        
        
    df_output = df_output[lst_final]

    return df_output


def use_data_prep_baseline(
    df: pd.DataFrame, target_to_impute: str = 'MasVnrArea', data_prep_model: str = 'data_prep_knn.pkl'
) -> pd.DataFrame:
    
    if 'SalePrice' in df.columns:
        df_prepared = df.drop(columns='SalePrice').copy()
    else:
        df_prepared = df.copy()
    
    # listing the indexes with null values.
    lst_nulls = df_prepared[(df_prepared[target_to_impute].isna())].index.tolist()

    # listing the indexes WITHOUT null values.
    lst_keep = [x for x in df_prepared.index if x not in lst_nulls]
    
    X = df_prepared.drop(columns=target_to_impute, index=lst_keep).copy()
    
#     Load model
#     TODO: Add the model as a parameter of this function.
#     model = Model.load(MODELS_FOLDER + 'data_prep_knn.pmml')
    model = pickle.load(open(MODELS_FOLDER + data_prep_model, 'rb'))
    y_pred = model.predict(X)
    
    df_prepared.loc[lst_nulls, 'MasVnrArea'] = y_pred     
    
    return df_prepared

## Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [4]:
RAW_FOLDER = '../data/raw/'
INTERIM_FOLDER = '../data/interim/'
REPORTS_FOLDER = '../reports/'
MODELS_FOLDER = '../models/'
RANDOM_STATE = 42


# Data import
We retrieve all the required data for the analysis.

In [5]:
df = pd.read_csv(RAW_FOLDER + 'train.csv', index_col=0)
df_evaluation = df.copy() 
df_evaluation.shape

(1460, 80)

# Data processing

## 1st experiment: the baseline
Here we will:

1. Drop the attributes with more than 80% nulls; and
1. Drop all non_numerical features.

### List of attributes to drop

In [6]:
lst_columns_null = serie_nulos(df_evaluation, corte=0.5).index.tolist()

lst_columns_null

4 atributos/features/campos possuem mais de 0.5 de valores nulos.


['PoolQC', 'MiscFeature', 'Alley', 'Fence']

### List of area attributes

In [7]:
lst_area = [x for x in df_evaluation.columns if ('area' in x.lower()) or ('sf' in x.lower())]

print(f"""There is/are {len(lst_area)} area attributes:
{lst_area}""")

There is/are 14 area attributes:
['LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea']


### List of time attributes

With these features, I will create new time features that express the difference between the YrSold and the each other time feature. For example:

- 'YrSold' - 'YearBuilt' = 'YrSold_YearBuilt' -> express the building's age.

The difference will not be calculated between 'YrSold' and 'MoSold' because there's no meaning in doing this. 'MoSold' can be used later to observe if there's a seasonality within a year.

In [8]:
lst_time = [x for x in df_evaluation.columns if ('yr' in x.lower()) or ('year' in x.lower())]
# After reading the data description, I realized that 'MoSold' is a time attribute too.

print(f"""There is/are {len(lst_time)} time attributes:
{lst_time}""")

lst_time.remove('YrSold')

There is/are 4 time attributes:
['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']


### List of float attributes

In [9]:
lst_float = [
    x for x in df_evaluation.select_dtypes(include='float64').columns.tolist() if (x not in lst_area) and (x not in lst_time)
]

print(f"""There is/are {len(lst_float)} float attributes:
{lst_float}""")

There is/are 1 float attributes:
['LotFrontage']


### Data output

In [10]:
lst_time

['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

In [11]:
df_output = df.copy()

df_output = prepare_baseline(df_output)

df_output.head()


,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,YrSold,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,
1,8450,196.0,706,0,150,856,856,854,0,1710,548,0,61,0,2008,5.0,5.0,5.0,208500
2,9600,0.0,978,0,284,1262,1262,0,0,1262,460,298,0,0,2007,31.0,31.0,31.0,181500
3,11250,162.0,486,0,434,920,920,866,0,1786,608,0,42,0,2008,7.0,6.0,7.0,223500
4,9550,0.0,216,0,540,756,961,756,0,1717,642,0,35,0,2006,91.0,36.0,8.0,140000
5,14260,350.0,655,0,490,1145,1145,1053,0,2198,836,192,84,0,2008,8.0,8.0,8.0,250000


### Presence of null values

The 1st experiment will be a simple linear regression. This method do not allowed null values in the dataset nor in production model. The following steps we'll fill the null values:

In [12]:
serie_nulos(df_output, corte=0)

1 atributos/features/campos possuem mais de 0 de valores nulos.


MasVnrArea    0.005479
dtype: float64

#### Filling attribute: *GarageYrBlt_YrSold*

This attribute could mean that the building has no garage. For this one we'll just fill the null values with a high number (e.g.: 999).

In [13]:
# This dataframe below shows there is NO information about the attributes with 'Garage' in the name.
df.loc[df['GarageYrBlt'].isna(), [x for x in df.columns if 'arage' in x]].sample(5, random_state=RANDOM_STATE)

,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond
Id,,,,,,,
529,NaN,NaN,NaN,0,0,NaN,NaN
40,NaN,NaN,NaN,0,0,NaN,NaN
387,NaN,NaN,NaN,0,0,NaN,NaN
534,NaN,NaN,NaN,0,0,NaN,NaN
288,NaN,NaN,NaN,0,0,NaN,NaN


In [14]:
# 2022-06-26 - I've added the fillna below in create_time_columns function.
df_output[['GarageYrBlt_YrSold']] = df_output[['GarageYrBlt_YrSold']].fillna(999)

df_output.loc[
    df_output['GarageYrBlt_YrSold']==999, [x for x in df_output.columns if 'arage' in x]
].sample(5, random_state=RANDOM_STATE)

,GarageArea,GarageYrBlt_YrSold
Id,,
529,0,999.0
40,0,999.0
387,0,999.0
534,0,999.0
288,0,999.0


#### Filling attribute: *MasVnrArea*

As showed above, this attribute or factor has only 8 null value entries (0,5% of total). As a first try, it is possible to delete these 8 entries, but we want to build a data prep pipeline to deal with these cases.

Here we are going to try [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) approach, without the target or response, obviously.

In [15]:
# Creating a copy of the df_output:
df_prep_MasVnrArea = df_output.copy()

# listing the indexes with null values.
lst_nulls_MasVnrArea = df_prep_MasVnrArea[(df_prep_MasVnrArea['MasVnrArea'].isna())].index.tolist()

# listing the indexes WITHOUT null values.
lst_keep_MasVnrArea = [x for x in df_prep_MasVnrArea.index if x not in lst_nulls_MasVnrArea]

df_prep_MasVnrArea[(df_prep_MasVnrArea['MasVnrArea'].isna())]

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,YrSold,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,
235,7851,NaN,625,0,235,860,860,1100,0,1960,440,288,48,0,2010,8.0,8.0,8.0,216500
530,32668,NaN,1219,0,816,2035,2515,0,0,2515,484,0,0,0,2007,50.0,32.0,32.0,200624
651,8125,NaN,0,0,813,813,822,843,0,1665,562,0,0,0,2008,1.0,1.0,1.0,205950
937,10083,NaN,833,0,343,1176,1200,0,0,1200,555,0,41,0,2009,6.0,6.0,6.0,184900
974,11639,NaN,0,0,1428,1428,1428,0,0,1428,480,0,120,0,2008,1.0,0.0,1.0,182000
978,4274,NaN,1106,0,135,1241,1241,0,0,1241,569,0,116,0,2007,1.0,0.0,0.0,199900
1244,13891,NaN,1386,0,690,2076,2076,0,0,2076,850,216,229,0,2006,0.0,0.0,0.0,465000
1279,9473,NaN,804,0,324,1128,1128,903,0,2031,577,0,211,0,2008,6.0,6.0,6.0,237000


In [16]:
X_train = df_prep_MasVnrArea.drop(columns=['MasVnrArea', 'SalePrice'], index=lst_nulls_MasVnrArea)
X_test = df_prep_MasVnrArea.drop(columns=['MasVnrArea', 'SalePrice'], index=lst_keep_MasVnrArea)
y_train = df_prep_MasVnrArea.loc[lst_keep_MasVnrArea, ['MasVnrArea']]
y_test = df_prep_MasVnrArea.loc[lst_nulls_MasVnrArea, ['MasVnrArea']]

print(f"""train: X - {X_train.shape}, y - {y_train.shape}
test: X - {X_test.shape}, y - {y_test.shape}""")

train: X - (1452, 17), y - (1452, 1)
test: X - (8, 17), y - (8, 1)


### Exporting the data prep model

#### Trying to use PMML

In [17]:
pipeline = PMMLPipeline([
    ("knn", KNeighborsRegressor(n_neighbors=5, n_jobs=-1))
])

pipeline.fit(X_train, y_train)

sklearn2pmml(pipeline, MODELS_FOLDER + 'data_prep_knn.pmml', with_repr = True)

y_pred_imputer = pipeline.predict(X_test)

y_pred_imputer

# df_output.loc[lst_nulls_MasVnrArea, 'MasVnrArea_pred'] = y_pred_imputer

# df_output.loc[lst_nulls_MasVnrArea, ['MasVnrArea', 'MasVnrArea_pred']]

array([[450. ],
       [ 97.4],
       [  0. ],
       [ 44.4],
       [114.2],
       [260.8],
       [587. ],
       [103.6]])

#### Using Pickle

In [18]:
pipeline = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)

pipeline.fit(X_train, y_train)

pickle.dump(pipeline, open(MODELS_FOLDER + 'data_prep_knn.pkl', 'wb'))

y_pred_imputer = pipeline.predict(X_test)

y_pred_imputer

array([[450. ],
       [ 97.4],
       [  0. ],
       [ 44.4],
       [114.2],
       [260.8],
       [587. ],
       [103.6]])

In [19]:
df_output.loc[lst_nulls_MasVnrArea, 'MasVnrArea'] = y_pred_imputer
              
df_output.loc[lst_nulls_MasVnrArea, :]

,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,YrSold,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,
235,7851,450.0,625,0,235,860,860,1100,0,1960,440,288,48,0,2010,8.0,8.0,8.0,216500
530,32668,97.4,1219,0,816,2035,2515,0,0,2515,484,0,0,0,2007,50.0,32.0,32.0,200624
651,8125,0.0,0,0,813,813,822,843,0,1665,562,0,0,0,2008,1.0,1.0,1.0,205950
937,10083,44.4,833,0,343,1176,1200,0,0,1200,555,0,41,0,2009,6.0,6.0,6.0,184900
974,11639,114.2,0,0,1428,1428,1428,0,0,1428,480,0,120,0,2008,1.0,0.0,1.0,182000
978,4274,260.8,1106,0,135,1241,1241,0,0,1241,569,0,116,0,2007,1.0,0.0,0.0,199900
1244,13891,587.0,1386,0,690,2076,2076,0,0,2076,850,216,229,0,2006,0.0,0.0,0.0,465000
1279,9473,103.6,804,0,324,1128,1128,903,0,2031,577,0,211,0,2008,6.0,6.0,6.0,237000


In [20]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   LotArea              1460 non-null   int64  
 1   MasVnrArea           1460 non-null   float64
 2   BsmtFinSF1           1460 non-null   int64  
 3   BsmtFinSF2           1460 non-null   int64  
 4   BsmtUnfSF            1460 non-null   int64  
 5   TotalBsmtSF          1460 non-null   int64  
 6   1stFlrSF             1460 non-null   int64  
 7   2ndFlrSF             1460 non-null   int64  
 8   LowQualFinSF         1460 non-null   int64  
 9   GrLivArea            1460 non-null   int64  
 10  GarageArea           1460 non-null   int64  
 11  WoodDeckSF           1460 non-null   int64  
 12  OpenPorchSF          1460 non-null   int64  
 13  PoolArea             1460 non-null   int64  
 14  YrSold               1460 non-null   int64  
 15  YearBuilt_YrSold     1460 non-null   f

### Conclusion

As we can see above, there is no null values. Now let's code our dataprep pipeline:

### Dataprep pipeline

In [21]:
# df_output.to_parquet(INTERIM_FOLDER + 'df_train_interim_01_baseline.pqt')

### Processing the Kaggle's test dataset

#### Importing

In [22]:
# Importing Kaggle's test dataset.
x_test_kaggle = pd.read_csv(RAW_FOLDER + 'test.csv', index_col='Id')
print(f"x_test_kaggle's shape: {x_test_kaggle.shape}")

x_test_kaggle's shape: (1459, 79)


#### Preprocessing

In [23]:
x_test_kaggle_prepared = prepare_baseline(x_test_kaggle)
x_test_kaggle_prepared[['GarageYrBlt_YrSold']] = x_test_kaggle_prepared[['GarageYrBlt_YrSold']].fillna(999)
x_test_kaggle_prepared = use_data_prep_baseline(x_test_kaggle_prepared)
x_test_kaggle_prepared.head()

# It is not correct, but I added a for loop here just to be sure that no null value will pass:
for column in x_test_kaggle_prepared.columns:
    if any(x_test_kaggle_prepared[column].isna()):
        x_test_kaggle_prepared[[column]] = x_test_kaggle_prepared[[column]].fillna(-999)

#### Saving Kaggle's test dataset

In [24]:
x_test_kaggle_prepared.to_parquet(INTERIM_FOLDER + 'x_test_kaggle_prepared_baseline.pqt')

## 2nd Data prep experiment: adding categorical attributes, first the binary ones

From the business and data understanding, I'd like to add some categorical attributes to the numerical ones added in the baseline dataset.

In this experiment, I'll use the float, time and binaries attributes.

Obviasly, the high null proportion attributes will not be used again ('PoolQC', 'MiscFeature', 'Alley', 'Fence').

In [8]:
# Importing the original dataset and excluding the high null proportion attributes:
lst_columns_null = ['PoolQC', 'MiscFeature', 'Alley', 'Fence']
df_evaluation = df.drop(columns=lst_columns_null).copy()

print(f"df_evaluation's shape: {df_evaluation.shape}")

df_evaluation's shape: (1460, 76)


### Int to Float

In [12]:
lst_area = [
    'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'SalePrice'
]

for column in lst_area:
    df_evaluation[column] = df[column].astype(float)
    
df_evaluation[lst_area].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotArea        1460 non-null   float64
 1   MasVnrArea     1452 non-null   float64
 2   BsmtFinSF1     1460 non-null   float64
 3   BsmtFinSF2     1460 non-null   float64
 4   BsmtUnfSF      1460 non-null   float64
 5   TotalBsmtSF    1460 non-null   float64
 6   1stFlrSF       1460 non-null   float64
 7   2ndFlrSF       1460 non-null   float64
 8   LowQualFinSF   1460 non-null   float64
 9   GrLivArea      1460 non-null   float64
 10  GarageArea     1460 non-null   float64
 11  WoodDeckSF     1460 non-null   float64
 12  OpenPorchSF    1460 non-null   float64
 13  PoolArea       1460 non-null   float64
 14  EnclosedPorch  1460 non-null   float64
 15  3SsnPorch      1460 non-null   float64
 16  ScreenPorch    1460 non-null   float64
 17  SalePrice      1460 non-null   float64
dtypes: float

### Binaries

In [13]:
df_evaluation['has_central_air'] = np.where(df_evaluation['CentralAir']=='Y', 1, 0)
df_evaluation['is_paved_street'] = np.where(df_evaluation['Street']=='Pave', 1, 0)
df_evaluation['is_all_pub_utilities'] = np.where(df_evaluation['Utilities']=='AllPub', 1, 0)

df_evaluation.drop(columns=['CentralAir', 'Street', 'Utilities'], inplace=True)

print(f"df_evaluation's shape: {df_evaluation.shape}")

df_evaluation's shape: (1460, 76)


### Creating time attributes

Using the custom function create_time_dif_columns created in the 1st experiment.

In [14]:
lst_time = [x for x in df_evaluation.columns if ('yr' in x.lower()) or ('year' in x.lower())]
# After reading the data description, I realized that 'MoSold' is a time attribute too.

print(f"""There is/are {len(lst_time)} time attributes:
{lst_time}""")

lst_time.remove('YrSold')

There is/are 4 time attributes:
['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']


In [15]:
df_2nd_experiment = df_evaluation.copy()

lst_final = ['has_central_air', 'is_paved_street', 'is_all_pub_utilities', 'YrSold', *lst_time, *lst_area]

df_2nd_experiment = df_2nd_experiment[lst_final]

df_2nd_experiment = create_time_dif_columns(df=df_2nd_experiment, lst=lst_time)

df_2nd_experiment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   has_central_air       1460 non-null   int32  
 1   is_paved_street       1460 non-null   int32  
 2   is_all_pub_utilities  1460 non-null   int32  
 3   YrSold                1460 non-null   int64  
 4   LotArea               1460 non-null   float64
 5   MasVnrArea            1452 non-null   float64
 6   BsmtFinSF1            1460 non-null   float64
 7   BsmtFinSF2            1460 non-null   float64
 8   BsmtUnfSF             1460 non-null   float64
 9   TotalBsmtSF           1460 non-null   float64
 10  1stFlrSF              1460 non-null   float64
 11  2ndFlrSF              1460 non-null   float64
 12  LowQualFinSF          1460 non-null   float64
 13  GrLivArea             1460 non-null   float64
 14  GarageArea            1460 non-null   float64
 15  WoodDeckSF           

#### Filling attribute: *MasVnrArea*

As showed above, this attribute or factor has only 8 null value entries (0,5% of total). As a first try, it is possible to delete these 8 entries, but we want to build a data prep pipeline to deal with these cases.

Here we are going to try [KNN](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) approach, without the target or response, obviously.

In [16]:
# Creating a copy of the df_output:
df_prep_MasVnrArea = df_2nd_experiment.copy()

# listing the indexes with null values.
lst_nulls_MasVnrArea = df_prep_MasVnrArea[(df_prep_MasVnrArea['MasVnrArea'].isna())].index.tolist()

# listing the indexes WITHOUT null values.
lst_keep_MasVnrArea = [x for x in df_prep_MasVnrArea.index if x not in lst_nulls_MasVnrArea]

df_prep_MasVnrArea[(df_prep_MasVnrArea['MasVnrArea'].isna())]

,has_central_air,is_paved_street,is_all_pub_utilities,YrSold,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,EnclosedPorch,3SsnPorch,ScreenPorch,SalePrice,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,
235,1,1,1,2010,7851.0,NaN,625.0,0.0,235.0,860.0,860.0,1100.0,0.0,1960.0,440.0,288.0,48.0,0.0,0.0,0.0,0.0,216500.0,8.0,8.0,8.0
530,1,1,1,2007,32668.0,NaN,1219.0,0.0,816.0,2035.0,2515.0,0.0,0.0,2515.0,484.0,0.0,0.0,0.0,200.0,0.0,0.0,200624.0,50.0,32.0,32.0
651,1,1,1,2008,8125.0,NaN,0.0,0.0,813.0,813.0,822.0,843.0,0.0,1665.0,562.0,0.0,0.0,0.0,0.0,0.0,0.0,205950.0,1.0,1.0,1.0
937,1,1,1,2009,10083.0,NaN,833.0,0.0,343.0,1176.0,1200.0,0.0,0.0,1200.0,555.0,0.0,41.0,0.0,0.0,0.0,0.0,184900.0,6.0,6.0,6.0
974,1,1,1,2008,11639.0,NaN,0.0,0.0,1428.0,1428.0,1428.0,0.0,0.0,1428.0,480.0,0.0,120.0,0.0,0.0,0.0,0.0,182000.0,1.0,0.0,1.0
978,1,1,1,2007,4274.0,NaN,1106.0,0.0,135.0,1241.0,1241.0,0.0,0.0,1241.0,569.0,0.0,116.0,0.0,0.0,0.0,0.0,199900.0,1.0,0.0,0.0
1244,1,1,1,2006,13891.0,NaN,1386.0,0.0,690.0,2076.0,2076.0,0.0,0.0,2076.0,850.0,216.0,229.0,0.0,0.0,0.0,0.0,465000.0,0.0,0.0,0.0
1279,1,1,1,2008,9473.0,NaN,804.0,0.0,324.0,1128.0,1128.0,903.0,0.0,2031.0,577.0,0.0,211.0,0.0,0.0,0.0,0.0,237000.0,6.0,6.0,6.0


In [17]:
X_train = df_prep_MasVnrArea.drop(columns=['MasVnrArea', 'SalePrice'], index=lst_nulls_MasVnrArea)
X_test = df_prep_MasVnrArea.drop(columns=['MasVnrArea', 'SalePrice'], index=lst_keep_MasVnrArea)
y_train = df_prep_MasVnrArea.loc[lst_keep_MasVnrArea, ['MasVnrArea']]
y_test = df_prep_MasVnrArea.loc[lst_nulls_MasVnrArea, ['MasVnrArea']]

print(f"""train: X - {X_train.shape}, y - {y_train.shape}
test: X - {X_test.shape}, y - {y_test.shape}""")

pipeline = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)
pipeline.fit(X_train, y_train)

pickle.dump(pipeline, open(MODELS_FOLDER + 'data_prep_knn_2nd_experiment.pkl', 'wb'))

y_pred_imputer = pipeline.predict(X_test)
y_pred_imputer

train: X - (1452, 23), y - (1452, 1)
test: X - (8, 23), y - (8, 1)


array([[450. ],
       [ 97.4],
       [  0. ],
       [ 44.4],
       [147. ],
       [260.8],
       [587. ],
       [103.6]])

### Creating pipeline

In [18]:
def use_2nd_data_prep(df: pd.DataFrame) -> pd.DataFrame:
    if 'SalePrice' in df.columns:
        df_evaluation = df.drop(columns='SalePrice').copy()
    else:
        df_evaluation = df.copy()

    lst_columns_null = ['PoolQC', 'MiscFeature', 'Alley', 'Fence']

    lst_area = [
        'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
        '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch'
    ]

    lst_time = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

    lst_final = [
        'has_central_air', 'is_paved_street', 'is_all_pub_utilities', 'YrSold',
        'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea', 'EnclosedPorch',
        '3SsnPorch', 'ScreenPorch', 'YearBuilt_YrSold', 'YearRemodAdd_YrSold', 
        'GarageYrBlt_YrSold'
    ]

    df_evaluation.drop(columns=lst_columns_null, inplace=True)

    for column in lst_area:
        df_evaluation[column] = df[column].astype(float)

    df_evaluation['has_central_air'] = np.where(df_evaluation['CentralAir']=='Y', 1, 0)
    df_evaluation['is_paved_street'] = np.where(df_evaluation['Street']=='Pave', 1, 0)
    df_evaluation['is_all_pub_utilities'] = np.where(df_evaluation['Utilities']=='AllPub', 1, 0)

    df_evaluation.drop(columns=['CentralAir', 'Street', 'Utilities'], inplace=True)

    return df_evaluation[[
        'has_central_air', 'is_paved_street', 'is_all_pub_utilities', 'YrSold',
        'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea', 'EnclosedPorch',
        '3SsnPorch', 'ScreenPorch', *lst_time
    ]]

In [19]:
df_teste = use_2nd_data_prep(df)
df_teste = create_time_dif_columns(df_teste, lst=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'])
df_teste = use_data_prep_baseline(df_teste, data_prep_model='data_prep_knn_2nd_experiment.pkl')

df_teste.sample(5, random_state=RANDOM_STATE)

,has_central_air,is_paved_street,is_all_pub_utilities,YrSold,LotArea,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,GarageArea,WoodDeckSF,OpenPorchSF,PoolArea,EnclosedPorch,3SsnPorch,ScreenPorch,YearBuilt_YrSold,YearRemodAdd_YrSold,GarageYrBlt_YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,
893,1,1,1,2006,8414.0,0.0,663.0,0.0,396.0,1059.0,1068.0,0.0,0.0,1068.0,264.0,192.0,0.0,0.0,0.0,0.0,0.0,43.0,3.0,43.0
1106,1,1,1,2010,12256.0,362.0,1032.0,0.0,431.0,1463.0,1500.0,1122.0,0.0,2622.0,712.0,186.0,32.0,0.0,0.0,0.0,0.0,16.0,15.0,16.0
414,1,1,1,2010,8960.0,0.0,0.0,0.0,1008.0,1008.0,1028.0,0.0,0.0,1028.0,360.0,0.0,0.0,0.0,130.0,0.0,0.0,83.0,60.0,83.0
523,1,1,1,2006,5000.0,0.0,399.0,0.0,605.0,1004.0,1004.0,660.0,0.0,1664.0,420.0,0.0,24.0,0.0,36.0,0.0,0.0,59.0,56.0,56.0
1037,1,1,1,2009,12898.0,70.0,1022.0,0.0,598.0,1620.0,1620.0,0.0,0.0,1620.0,912.0,228.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0


#### Spliting and exporting training and validation datasets

In [20]:
df_output = pd.concat([df_teste, df[['SalePrice']]], axis=1)
df_output.shape

(1460, 25)

In [21]:
df_output.to_parquet(INTERIM_FOLDER + 'df_train_interim_02_2nd_experiment.pqt')

In [22]:
X = df.drop(columns=['SalePrice'])
y = df[['SalePrice']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

### Processing the Kaggle's test dataset

#### Importing

In [23]:
# Importing Kaggle's test dataset.
x_test_kaggle = pd.read_csv(RAW_FOLDER + 'test.csv', index_col='Id')
print(f"x_test_kaggle's shape: {x_test_kaggle.shape}")

x_test_kaggle's shape: (1459, 79)


#### Preprocessing

In [24]:
x_test_kaggle_prepared = use_2nd_data_prep(x_test_kaggle)

x_test_kaggle_prepared = create_time_dif_columns(
    x_test_kaggle_prepared, lst=['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
)

x_test_kaggle_prepared = use_data_prep_baseline(x_test_kaggle_prepared, data_prep_model='data_prep_knn_2nd_experiment.pkl')

# It is not correct, but I added a for loop here just to be sure that no null value will pass:
for column in x_test_kaggle_prepared.columns:
    if any(x_test_kaggle_prepared[column].isna()):
        x_test_kaggle_prepared[[column]] = x_test_kaggle_prepared[[column]].fillna(-999)

x_test_kaggle_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   has_central_air       1459 non-null   int32  
 1   is_paved_street       1459 non-null   int32  
 2   is_all_pub_utilities  1459 non-null   int32  
 3   YrSold                1459 non-null   int64  
 4   LotArea               1459 non-null   float64
 5   MasVnrArea            1459 non-null   float64
 6   BsmtFinSF1            1459 non-null   float64
 7   BsmtFinSF2            1459 non-null   float64
 8   BsmtUnfSF             1459 non-null   float64
 9   TotalBsmtSF           1459 non-null   float64
 10  1stFlrSF              1459 non-null   float64
 11  2ndFlrSF              1459 non-null   float64
 12  LowQualFinSF          1459 non-null   float64
 13  GrLivArea             1459 non-null   float64
 14  GarageArea            1459 non-null   float64
 15  WoodDeckSF        

#### Saving Kaggle's test dataset

In [133]:
x_test_kaggle_prepared.to_parquet(INTERIM_FOLDER + 'x_test_kaggle_prepared_2nd_experiment.pqt')

In [25]:
x_test_kaggle.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## 3rd experiment: Using floats, integer and binary attributes

In [63]:
# Importing the original dataset and excluding the high null proportion attributes:
lst_columns_null = ['PoolQC', 'MiscFeature', 'Alley', 'Fence']
df_evaluation = df.drop(columns=lst_columns_null).copy()

print(f"df_evaluation's shape: {df_evaluation.shape}")

df_evaluation's shape: (1460, 76)


### Tranforming Int to Float

In [74]:
lst_area = [
    'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
    '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'MiscVal'
]

for column in lst_area:
    df_evaluation[column] = df[column].astype(float)

#### Imputing data in MasVnrArea

In [75]:
imputer = MeanMedianImputer(imputation_method='median', variables=lst_area)

df_evaluation = imputer.fit_transform(df_evaluation)

### Integers

In [76]:
lst_int = [
    'BsmtHalfBath', 'HalfBath', 'BsmtFullBath', 'Fireplaces', 'FullBath', 'KitchenAbvGr',
    'GarageCars', 'BedroomAbvGr', 'OverallCond', 'OverallQual', 'TotRmsAbvGrd'
]

### Binaries

In [77]:
# set up the encoder
encoder = OneHotEncoder(top_categories=2, variables=['CentralAir', 'Street', 'Utilities'], drop_last_binary=True)

# fit the encoder
df_test_transformed = encoder.fit_transform(df_evaluation)

print(f"df_test_transformed's shape: {df_test_transformed.shape}")

lst_new_columns = [
    x for x in df_test_transformed.columns if x not in df_evaluation.columns
]

df_test_transformed[lst_new_columns].head(3)

df_test_transformed's shape: (1460, 76)


,CentralAir_Y,Street_Pave,Utilities_AllPub
Id,,,
1,1,1,1
2,1,1,1
3,1,1,1


### Creating time atributes

In [78]:
lst_time = [x for x in df_evaluation.columns if ('yr' in x.lower()) or ('year' in x.lower())]
# After reading the data description, I realized that 'MoSold' is a time attribute too.

print(f"""There is/are {len(lst_time)} time attributes:
{lst_time}""")

lst_time.remove('YrSold')

There is/are 4 time attributes:
['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']


### Spliting data

In [113]:
X = df.drop(columns=[*lst_columns_null, 'SalePrice']).copy()
y = df[['SalePrice']].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

X_train = create_time_dif_columns(df=X_train, lst=lst_time)
X_test = create_time_dif_columns(df=X_test, lst=lst_time)

Pipe = Pipeline([
    ('categorical_imputer', CategoricalImputer(variables=['CentralAir', 'Street', 'Utilities'])), 
    ('binary', OneHotEncoder(top_categories=2, variables=['CentralAir', 'Street', 'Utilities'], drop_last_binary=True)),
    ('median_imputer', MeanMedianImputer(imputation_method='median', variables=[*lst_area, *lst_int]))
])

X_train = Pipe.fit_transform(X_train)
X_test = Pipe.transform(X_test)

lst_final = [
    *lst_new_columns, 'YrSold', 'YearBuilt_YrSold', 'YearRemodAdd_YrSold', 'GarageYrBlt_YrSold', *lst_area, *lst_int
]

X_train = X_train[lst_final]
X_test = X_test[lst_final]

X_train.to_parquet(INTERIM_FOLDER + 'X_train_3rd_dataprep.pqt')
X_test.to_parquet(INTERIM_FOLDER + 'X_test_3rd_dataprep.pqt')
y_train.to_parquet(INTERIM_FOLDER + 'y_train_3rd_dataprep.pqt')
y_test.to_parquet(INTERIM_FOLDER + 'y_test_3rd_dataprep.pqt')

X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 255 to 1127
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CentralAir_Y         1168 non-null   int32  
 1   Street_Pave          1168 non-null   int32  
 2   Utilities_AllPub     1168 non-null   int32  
 3   YrSold               1168 non-null   int64  
 4   YearBuilt_YrSold     1168 non-null   float64
 5   YearRemodAdd_YrSold  1168 non-null   float64
 6   GarageYrBlt_YrSold   1168 non-null   float64
 7   LotArea              1168 non-null   int64  
 8   MasVnrArea           1168 non-null   float64
 9   BsmtFinSF1           1168 non-null   int64  
 10  BsmtFinSF2           1168 non-null   int64  
 11  BsmtUnfSF            1168 non-null   int64  
 12  TotalBsmtSF          1168 non-null   int64  
 13  1stFlrSF             1168 non-null   int64  
 14  2ndFlrSF             1168 non-null   int64  
 15  LowQualFinSF         1168 non-null  

## Processing the kaggle's test dataset

In [114]:
# Importing Kaggle's test dataset.
x_test_kaggle = pd.read_csv(RAW_FOLDER + 'test.csv', index_col='Id')
print(f"x_test_kaggle's shape: {x_test_kaggle.shape}")

x_test_kaggle's shape: (1459, 79)


In [115]:
x_test_kaggle_prepared = x_test_kaggle.drop(columns=[*lst_columns_null]).copy()

x_test_kaggle_prepared = create_time_dif_columns(df=x_test_kaggle_prepared, lst=lst_time)

x_test_kaggle_prepared = Pipe.transform(x_test_kaggle_prepared)

lst_final = [
    *lst_new_columns, 'YrSold', 'YearBuilt_YrSold', 'YearRemodAdd_YrSold', 'GarageYrBlt_YrSold', *lst_area, *lst_int
]

x_test_kaggle_prepared = x_test_kaggle_prepared[lst_final]

x_test_kaggle_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 1461 to 2919
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CentralAir_Y         1459 non-null   int32  
 1   Street_Pave          1459 non-null   int32  
 2   Utilities_AllPub     1459 non-null   int32  
 3   YrSold               1459 non-null   int64  
 4   YearBuilt_YrSold     1459 non-null   float64
 5   YearRemodAdd_YrSold  1459 non-null   float64
 6   GarageYrBlt_YrSold   1459 non-null   float64
 7   LotArea              1459 non-null   int64  
 8   MasVnrArea           1459 non-null   float64
 9   BsmtFinSF1           1459 non-null   float64
 10  BsmtFinSF2           1459 non-null   float64
 11  BsmtUnfSF            1459 non-null   float64
 12  TotalBsmtSF          1459 non-null   float64
 13  1stFlrSF             1459 non-null   int64  
 14  2ndFlrSF             1459 non-null   int64  
 15  LowQualFinSF         1459 non-null 

In [116]:
x_test_kaggle_prepared.to_parquet(INTERIM_FOLDER + 'x_test_kaggle_3rd_dataprep.pqt')